# AI Club Evolutionary Algorithm Workshop

## Lets start with a couple of questions
- "So what the heck is an evolutionary algorithm?"
  - There are many types of programs that fall into the 'evolutionary algorithm' category. It is the umbrella term for all computational models that are inspired by evolutionary mechanisms. The most important implementations are Genetic Algorithms (GAs), Genetic Programming (GP), Evolutionary Programming (EP), and Classifier Systems and Artificial Life (AL); but these distinctions are not important for what we're doing
- "What's the goal?"
  - In EAs we have a fitness function that measures how well an individual performs, more fit individuals are seen as better. When we describe our fitness functions we usually do it as a measure of how well our goal was acheived. Therefore we can evolve our algorithm to do anything we can measure.
- "How does it work?"
  - ![title](EA.png)
  - Note that we are leaving out crossover for simplicity
- For more detailed explanations, check out these links
  - https://www.kip.uni-heidelberg.de/vision/previous-projects/evolvable-hardware/evolutionary-algorithms/
  - https://en.wikipedia.org/wiki/Evolutionary_algorithm
  - https://towardsdatascience.com/introduction-to-evolutionary-algorithms-a8594b484ac


We'll start by importing some important libraries we'll be needing
- tkinter - A GUI framework that we'll be using to display our algorithm
  - tkinter is simple and typically used for forms and simple guis, a more popular option is pygame
- math - this gives us extra functions for trig 
- random - this gives us capabilities to generate random numbers<br/><br/>
All of these libraries are part of the python standard library so no installs should be required

In [1]:
import tkinter as tk
from tkinter.font import BOLD, Font
import math
import random

First lets go ahead and make ourselves a function to help us later on

We'll need to be able to find which piece of food is closest to a bug at any time<br/>
Therefore, we should be able to take in a position, and with a list of objects, find the one that is closes

In [2]:
# In python, we can create variables to hold functions, this is really nice for things like common math functions
# This variable will hold a calculation for euclidian distance between points
euclid_dist = lambda pos1, pos2: ((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2) ** (1/2)
get_angle = lambda pos1, pos2: math.radians(math.atan2(pos1[1] - pos2[1], pos1[0] - pos2[0]))

def get_closest(pos, objs):
    #euclid_dist = lambda pos1, pos2: abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
    # We can say the first object in our list is the closest one for now
    nearest = objs[0]
    nearest_distance = euclid_dist(pos, nearest.pos)

    # Then, for all the objects in our list, we want to check if their euclidian distance is closer than all the previous objects
    # If it is, then we set the nearest object to the new object, and return the closest one
    for obj in objs:
        distance = euclid_dist(pos, obj.pos)
        if distance < nearest_distance:
            nearest_distance = distance
            nearest = obj
    return nearest

Next we'll be making classes for our "Food" and "Bug"s</br>
We'll be starting with the food class as the Bug class will depend on it</br>

- In python, the constructor of a class is always the function init pre and postceeded by two underscores
- Our food will have 
  - A point value (to give to the bugs when they 'eat' the food)
  - A position on our playing field
  - A shape associated with them so that we can draw it onto our screen
  - A canvas associated with them so that we can draw the shape onto it
- The hide function will act as a way to remove the food from the screen

In [3]:
class Food:
    def __init__(self, points, pos, oval, canvas):
        self.points = points
        self.pos = pos
        self.shape = oval
        self.canvas = canvas

    def hide(self):
        self.canvas.delete(self.shape)

Next we'll make the bug class, this one is a little more complicated so the descriptions will be in comments in the cell block below

In [4]:
class Bug:
    # Just like the food, we'll need to create a constructor for the bugs
    # However, we also now have a speed attribute for how fast the bugs can move
    # This attribute is what we will 'evolve' with our algorithm
    def __init__(self, pos, speed, shape, canvas):
        self.pos = pos
        self.speed = speed
        self.shape = shape
        self.canvas = canvas
        self._score = 0

    # In python when we want to mark a variable as private we give it an understore at the start of it's name
    # Then if we want to get or set that variable, we create functions to access it
    # Note that the variable isn't actually private, we just expect people to treat it as such
    # It's also very uncommon to use setters and getters in python, we're just doig it here to demonstrate how private variables are done
    def get_score(self):
        return self._score

    def add_points(self, points):
        self._score += points

    def hide(self):
        self.canvas.delete(self.shape)

    def move(self, food_list):
        if len(food_list) > 0:
            nearest = get_closest(self.pos, food_list)
            # If the distance to a piece of food is less than our speed, lets eat the food
            dist = euclid_dist(self.pos, nearest.pos)
            if dist <= self.speed:
                x = (nearest.pos[0] - self.pos[0]) / dist
                y = (nearest.pos[1] - self.pos[1]) / dist
                self.canvas.move(self.shape, x, y)
                self.pos = (self.pos[0]+x, self.pos[1]+y)
                self.add_points(nearest.points)
                return nearest
            # If not, move towards the food
            else:
                dist = dist
                x = self.speed * (nearest.pos[0] - self.pos[0]) / dist
                y = self.speed * (nearest.pos[1] - self.pos[1]) / dist

                self.canvas.move(self.shape, x, y)
                self.pos = (self.pos[0]+x, self.pos[1]+y)


In order to evlove our population we'll need to collect all of the bugs with the highest scores

So we'll sort them by score and grab the top however many we want to keep (4 in this case)

In [5]:
def get_best_bugs(bugs):
    best = []
    scores = []
    for bug in bugs:
        scores += [bug.get_score()]

    scores.sort(reverse=True)
    scores = scores[:10]

    for bug in bugs:
        if bug.get_score() in scores and len(best) < 4:
            best += [bug]

    return best

This will help us remove all of the old food and bugs when we go to our next generation

In [6]:
def hide_all(bugs, foods, window):
    for bug in bugs:
        bug.hide()

    for food in foods:
        food.hide()

    window.update()

This will generate a new piece of food at a random location for us

In [7]:
def new_food(food_num, food_list, canvas):
    for i in range(food_num):
        radius = 5
        pos = (random.random()*1600, random.random()*900)
        oval=canvas.create_oval(pos[0]-radius, pos[1]-radius, pos[0]+radius, pos[1]+radius, fill='green')
        food_list += [Food(1, pos, oval, canvas)]

    canvas.pack()

This function will create a new generation for us<br/>
If we provide a set of best_bugs then we know to mutate them, if not then this must be the first generation and we'll need to initialize everything

In [8]:
def new_gen(canvas, win, best_bugs=None):
    radius = 5
    mutation_rate = 1.5
    bugs = []
    food = []

    new_food(600, food, canvas)

    if best_bugs is None:
        for i in range(20):
            pos = (0, random.random()*900)
            speed = random.random()*1
            oval=canvas.create_oval(pos[0]-radius, pos[1]-radius, pos[0]+radius, pos[1]+radius, fill='blue')
            bugs += [Bug(pos, speed, oval, canvas)]
        canvas.pack()
    else:
        speeds = []
        for bug in best_bugs:
            pos = (0, random.random()*900)
            speed = bug.speed
            speeds += [speed]
            oval=canvas.create_oval(pos[0]-radius, pos[1]-radius, pos[0]+radius, pos[1]+radius, fill='blue')
            bugs += [Bug(pos, speed, oval, canvas)]

        # Display some stats and whatnot
        av_speed = sum(speeds)/len(speeds)
        font = Font(win, size=15)
        label = tk.Label(win, text="Avg top 20% speed: {:.2f}".format(av_speed), font=font)
        label.place(x=1, y=100)

        # Create 20 new bugs that are mutated off of the best ones
        for i in range(20-(len(bugs))):
            pos = (0, random.random()*900)
            # This is our simplistic mutation function
            speed =  av_speed + random.random()*mutation_rate - random.random()*mutation_rate
            oval=canvas.create_oval(pos[0]-radius, pos[1]-radius, pos[0]+radius, pos[1]+radius, fill='blue')
            bugs += [Bug(pos, speed, oval, canvas)]

        canvas.pack()
    return bugs, food

This function will move us ahead one point in time (so move the bugs a little bit and check for collisions)

In [9]:
def tick(win, bugs, food_list):
    for b in bugs:
        ret_val = b.move(food_list)
        if ret_val != None:
            food_list.remove(ret_val)
            ret_val.hide()

    win.update()

In [10]:
# You can figure out this one, I believe in you
def run_generation(win, bugs, food_list, canvas):
    while len(food_list) > 0:
        tick(win, bugs, food_list)

Finally we can put everything together and start watching our bugs evolve (if you run this cell and don't see anything, check your taskbar for a new window)

In [11]:
win = tk.Tk()
generations = 30

canvas = tk.Canvas(win, width=1600, height=900)

font = Font(win, size=35, weight=BOLD)

while True:
    bugs, food = new_gen(canvas, win)

    canvas.pack()

    for i in range(generations):
        label = tk.Label(win, text="Gen "+str(i+1)+"/" + str(generations), font=font)
        label.place(x=1, y=1)
        run_generation(win, bugs, food, canvas)
        best_bugs = get_best_bugs(bugs)
        hide_all(bugs, food, win)
        bugs, food = new_gen(canvas, win, best_bugs=best_bugs)

    hide_all(bugs, food, win)

TclError: invalid command name ".!canvas"

# Challenges
Be sure to show the challenges you complete to an eboard member to get challenge tickets<br/>
The challenges are each worth 1, 2, and 3 tickets, respectively
## Easy 
- In the articles linked at the start of this workshop, you may have noticed that they talk about something called crossover. Crossover helps take genes from multiple of the best bugs and combining them to get a mutation. Currently we are mutating direct copies of the parents, can you come up with a way to combine genes from the parents to create more unique offspring?
## Medium
- Right now we have only a single fitness function as the Bugs only need to collect food. However, you may have noticed in the three articles linked at the start of this notebook (I particularly like the third ones explanation), that we can have multiple fitness functions for different objectives.
  - What if we added another type of resource that they needed to collect? (ie, water). Can you implement a new type of resource, with a separate score and fitness function, and include that in the evolution of the Bugs? (How can we have this help create new crossover combinations? Or even more unique mutations?)
## Hard
- In evolutionary algorithms the parents with the best fitness can sometimes be flukes who just got lucky, therefore we can introduce a retention where we keep at least one offspring from different families. In the first evolution, we have at least two members of each family, then the best performing families gain another, and the worst performing families lose a member in proportional amounts (ie, if we gain 3 total we also lose 3 total).
  - Can you implement a system where the bugs are put into families and make it so that families never completely die off?
  - In order to see that families never die off, we should cap how far a specific familiy can evolve (ie, generate a random cap on the speed they can evolve to between 2 and 40)